In [64]:
import cv2
import numpy
import numpy as np
from PIL import Image
import torch
from pathlib import Path
from moviepy.editor import VideoFileClip
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [65]:
train_dir=r"c:\Users\61910253\Desktop\sign language\Indian"

In [66]:
import os
files=os.listdir(fldr)

In [67]:
labels = sorted(os.listdir(fldr))

In [68]:
img_width, img_height = 256, 256
cap = cv2.VideoCapture('my_video.mp4')

In [69]:
ret, frame = cap.read()

In [70]:
video = VideoFileClip('resource/data/hello.mp4')

In [72]:
video_path = Path(r"c:\Users\61910253\Desktop\sign language\resource\data\How are you.mp4")
cap = cv2.VideoCapture(str(video_path))


In [73]:
while True:
    ret, frame = cap.read()
    if not ret:
        break



In [74]:
batch_size = 64

In [75]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  fldr,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  fldr,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 42745 files belonging to 35 classes.
Using 29922 files for training.
Found 42745 files belonging to 35 classes.
Using 12823 files for validation.


In [76]:
class_names = train_ds.class_names
print(class_names)


['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [77]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 256, 256, 3)
(64,)


In [78]:
import tensorflow as tf

normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)


In [79]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0019607844 0.9546569


In [80]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [81]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers

In [82]:
num_classes = 24

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [83]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [84]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_5 (Rescaling)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     8,388,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,415,416 (32.10 MB)

 Trainable params: 8,415,416 (32.10 MB)

 Non-trainable params: 0 (0.00 B)

In [85]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

In [86]:
model = Sequential()

In [87]:
fle_s='Sign_Language_Detection.keras'
checkpoint=ModelCheckpoint(fle_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75,monitor='val_loss',restore_best_weights='True')
callbacks_list=[checkpoint,Early_stop]

In [88]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [89]:
train_images = []
for filename in os.listdir(train_dir):
    if filename.endswith(".jpg"):
        img = cv2.imread(os.path.join(train_dir, filename))
        train_images.append(img)



In [90]:
train_images = np.array(train_images)

In [91]:
train_dir = r"c:\Users\61910253\Desktop\sign language\Indian"
val_dir = r"c:\Users\61910253\Desktop\sign language\Indian"

In [98]:
# Assume that you have your training images and labels in lists
train_images_list = [...]  # list of training images
train_labels_list = [...]  # list of training labels

In [99]:

# Convert the lists to NumPy arrays
train_images = np.array(train_images_list)
train_labels = np.array(train_labels_list)

In [100]:
# Save the NumPy arrays to files
np.save('images.npy', train_images)
np.save('labels.npy', train_labels)

In [ ]:
train_dir = 'c:\\Users\\61910253\\oneDrive\\Desktop\\sign language\\Indian'